In [1]:
import requests
import json
import ssl
import pandas as pd

#(확인필요) 필요정보 시작페이지 4~168 * 1000 + 659건 = 164659건 
#<Response [200]> 성공시에만 데이터 처리 추가

tg_url = 'https://seoul.openapi.redtable.global/api/rstr' #식당기본정보,엔드포인트 URL
params = {
    'serviceKey': 'oZFJFbDm058mzGfbOGbVKGkpeZPJqRwu9vguWe7dpZuWoG7hRCtpEEroFjpnFY4w',
    'pageNo': 1 ,  # 시작 페이지
    'per_page': 1000  # 페이지당 데이터 개수
}

dfs = []

for page in range(1, 51): #대략 5초소요, 총 168페이지
    response = requests.get(tg_url, params=params)
    data     = json.loads(response.text)
    df       = pd.json_normalize(data['body'])
    dfs.append(df)
    
    params['pageNo'] += 1

total_df = pd.concat(dfs)
total_df.to_csv('./svData/restaurant.csv', index=False, encoding = 'CP949')

In [2]:
rdf = pd.read_csv('./svData/restaurant.csv', encoding = 'CP949')
rdf.shape

(50000, 11)

In [3]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RSTR_ID              50000 non-null  int64  
 1   RSTR_NM              50000 non-null  object 
 2   RSTR_RDNMADR         49474 non-null  object 
 3   RSTR_LNNO_ADRES      49004 non-null  object 
 4   RSTR_LA              50000 non-null  float64
 5   RSTR_LO              50000 non-null  float64
 6   RSTR_TELNO           46173 non-null  object 
 7   BSNS_STATM_BZCND_NM  47261 non-null  object 
 8   BSNS_LCNC_NM         50000 non-null  object 
 9   RSTR_INTRCN_CONT     50000 non-null  object 
 10  RSTR_AREA_CLSF_NM    2753 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 4.2+ MB


In [4]:
rdf.head()

,RSTR_ID,RSTR_NM,RSTR_RDNMADR,RSTR_LNNO_ADRES,RSTR_LA,RSTR_LO,RSTR_TELNO,BSNS_STATM_BZCND_NM,BSNS_LCNC_NM,RSTR_INTRCN_CONT,RSTR_AREA_CLSF_NM
0,1088,펜앤커피,서울특별시 종로구 인사동5길 12,서울특별시 종로구 관훈동 198-10,37.572857,126.985577,02-725-0062,기타,일반음식점,"서울특별시 종로구에서 가 볼 만한 식당을 찾으신다면? 종각역 근처에 있는 ""펜앤커피...",NaN
1,1097,배초향,서울특별시 강남구 도산대로55길 20,서울특별시 강남구 청담동 88-5,37.524788,127.040623,NaN,한식,일반음식점,"서울특별시 강남구에 가신다면 ""배초향""에 방문해보세요~",NaN
2,1115,토담토담,서울특별시 종로구 수표로 121,서울특별시 종로구 낙원동 218-1,37.571998,126.988855,02-744-7934,분식,일반음식점,"""토담토담""은 서울특별시 종로구에 있는 맛집으로, 가장 가까운 지하철역은 종로3가역...",NaN
3,1116,또와참숯돼지갈비,서울특별시 종로구 돈화문로11가길 48,서울특별시 종로구 와룡동 166,37.574727,126.990652,02-6760-0399,한식,일반음식점,"""또와참숯돼지갈비""는 서울특별시 종로구에 있는 맛집으로, 가장 가까운 지하철역은 안...",NaN
4,1117,동성각,서울특별시 종로구 새문안로9길 29-2,서울특별시 종로구 당주동 18,37.571444,126.975272,02-735-0107,중국식,일반음식점,"""동성각""은 서울특별시 종로구에 있는 맛집으로, 가장 가까운 지하철역은 광화문역입니...",NaN


In [5]:
rdf['BSNS_STATM_BZCND_NM'].unique()

array(['기타', '한식', '분식', '중국식', '경양식', '다방', nan, '일식', '커피숍', '호프/통닭',
       '외국음식전문점(인도,태국등)', '뷔페식', '제과점영업', '일반조리판매', '정종/대포집/소주방',
       '식육(숯불구이)', '김밥(도시락)', '패스트푸드', '탕류(보신용)', '기타 휴게음식점', '냉면집',
       '감성주점', '회집', '복어취급', '라이브카페', '관광호텔', '백화점', '기타(편의점)', '공항',
       '아이스크림', '철도역구내', '유원지', '전통찻집', '패밀리레스토랑'], dtype=object)

In [10]:
rdf['BSNS_STATM_BZCND_NM'].isnull().sum()  #nan 값 삭제

2739

In [11]:
rdf.dropna(subset=['BSNS_STATM_BZCND_NM'], inplace=True)

In [13]:
rdf.shape

(47261, 11)

In [18]:
rdf['RSTR_RDNMADR'].isnull().sum() #RSTR_RDNMADR 널값 503 삭제 < RSTR_LNNO_ADRES 널값 946 
#rdf.dropna(subset=['RSTR_RDNMADR'], inplace=True)

0

In [19]:
rdf['gu'] = rdf['RSTR_RDNMADR'].apply(lambda x: x.split(' ')[1] if isinstance(x, str) and len(x.split(' ')) > 1 else '')

In [21]:
#rdf.head()
rdf['gu'].isnull().sum()

0

In [22]:
rdf.to_csv('./svData/restaurant.csv', index = False, encoding = 'CP949') #최종 저장

In [23]:
rdf.shape

(46758, 12)